In [1]:
!pwd

/home/giakhang/dev/multi_task_autopilot/YOLOF-Mask/test


In [2]:
%cd /home/giakhang/dev/multi_task_autopilot/YOLOF-Mask

/home/giakhang/dev/multi_task_autopilot/YOLOF-Mask


In [3]:
import sys

import argparse
import sys
import os
import torch, detectron2

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets.coco_panoptic import register_coco_panoptic_separated
from detectron2.model_zoo import get_config
from detectron2.config import LazyConfig
from detectron2.config.instantiate import instantiate
from detectron2.engine import default_setup

from tools.lazyconfig_train_net import do_train

In [4]:
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
#os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [5]:
BDD100K_IMG_DIR = "/home/giakhang/dev/multi_task_autopilot/dataset_zoo/bdd100k/images/10k"
DATASET_NAME = "bdd100k"
DETECTRON2_ANNOT_DIR = "/home/giakhang/dev/multi_task_autopilot/da-panopticfpn/datasets/bdd100k_reduced"

PANOPTIC_ROOT = os.path.join(DETECTRON2_ANNOT_DIR, "bdd100k_panoptic_reduced")

for split in ['train', 'val']:
    d_name = DATASET_NAME + f'_{split}'
    register_coco_panoptic_separated(
        d_name, 
        {}, 
        image_root=os.path.join(BDD100K_IMG_DIR, split),
        panoptic_root=os.path.join(PANOPTIC_ROOT, split),
        panoptic_json=os.path.join(DETECTRON2_ANNOT_DIR, f"bdd100k_panoptic_reduced_{split}.json"),
        sem_seg_root=os.path.join(DETECTRON2_ANNOT_DIR, f"{split}_sem_stuff"),
        instances_json=os.path.join(DETECTRON2_ANNOT_DIR, f"bdd100k_instances_reduced_{split}.json"),
    )

STUFF_CLASSES = ["unlabeled", "dynamic", "ego vehicle", "ground", "static",  
    "parking", "rail track", "road", "sidewalk", "bridge", 
    "building", "fence", "garage", "guard rail", "tunnel", 
    "wall", "banner", "billboard", "lane divider", "parking sign", 
    "pole", "polegroup", "street light", "traffic cone",  
    "traffic device", "traffic light", "traffic sign",  
    "traffic sign frame", "terrain", "vegetation", "sky"]

MetadataCatalog.get("bdd100k_train_separated").stuff_classes = STUFF_CLASSES
MetadataCatalog.get("bdd100k_val_separated").stuff_classes = STUFF_CLASSES

In [6]:
config_file = "/home/giakhang/dev/multi_task_autopilot/YOLOF-Mask/configs/PanopticSegmentation/panoptic_yolof_mask_R_50_1x.py"

In [ ]:
class Args(argparse.Namespace):
    config_file=config_file
    eval_only=False
    num_gpus=1
    num_machines=1
    resume=False

args = Args()

cfg = LazyConfig.load(config_file)

default_setup(cfg, args)

do_train(args, cfg)

[11/11 11:33:32 detectron2]: Rank of current process: 0. World size: 1
[11/11 11:33:32 detectron2]: Rank of current process: 0. World size: 1
[11/11 11:33:32 detectron2]: Environment info:
-------------------------------  -----------------------------------------------------------------------------------------------------------------------------------------------------------------
sys.platform                     linux
Python                           3.10.15 (main, Oct  3 2024, 07:27:34) [GCC 11.2.0]
numpy                            1.26.4
detectron2                       0.6 @/home/giakhang/dev/multi_task_autopilot/detectron2/detectron2
detectron2._C                    not built correctly: /home/giakhang/dev/multi_task_autopilot/detectron2/detectron2/_C.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN3c104cuda9SetDeviceEi
Compiler ($CXX)                  c++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
CUDA compiler                    Build cuda_11.8.r11.8/compiler.31833905_0
detectro

/home/giakhang/anaconda3/envs/yolof_mask/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[11/11 11:33:50 d2.utils.events]:  eta: 1 day, 19:53:03  iter: 19  total_loss: 4.673  loss_sem_seg: 2.058  loss_cls: 1.313  loss_box_reg: 1.313  loss_mask: 0    time: 0.7332  last_time: 0.6725  data_time: 0.0391  last_data_time: 0.0275   lr: 5.4135e-05  max_mem: 5774M
[11/11 11:33:50 d2.utils.events]:  eta: 1 day, 19:53:03  iter: 19  total_loss: 4.673  loss_sem_seg: 2.058  loss_cls: 1.313  loss_box_reg: 1.313  loss_mask: 0    time: 0.7332  last_time: 0.6725  data_time: 0.0391  last_data_time: 0.0275   lr: 5.4135e-05  max_mem: 5774M
[11/11 11:34:03 d2.utils.events]:  eta: 1 day, 19:58:07  iter: 39  total_loss: 4.334  loss_sem_seg: 1.583  loss_cls: 1.241  loss_box_reg: 1.282  loss_mask: 0    time: 0.6972  last_time: 0.6859  data_time: 0.0236  last_data_time: 0.0228   lr: 0.0001041  max_mem: 5774M
[11/11 11:34:03 d2.utils.events]:  eta: 1 day, 19:58:07  iter: 39  total_loss: 4.334  loss_sem_seg: 1.583  loss_cls: 1.241  loss_box_reg: 1.282  loss_mask: 0    time: 0.6972  last_time: 0.6859  